In [1]:
# pip install --user gensim

In [2]:
import pandas as pd
import sklearn.feature_extraction as fe
# import sklearn.metrics as m
from gensim.models import Word2Vec
from score_F1 import score_F1
import string
import numpy as np

#
import pickle
import json
import os



Notebook-only

In [3]:
#
original_directory = os.getcwd()
os.chdir("../../data/raw")
os.listdir()

['20News_coarse.pkl',
 'nyt_seedwords.json',
 '20News_seedwords.json',
 'nyt_coarse.pkl']

In [4]:
# read data

#
with open("nyt_coarse.pkl", "rb") as fp:
    data_nyt = pickle.load(fp)
with open("20News_coarse.pkl", "rb") as fp:
    data_20news = pickle.load(fp)
    
with open("nyt_seedwords.json") as fp:
    seed_nyt = json.load(fp)
with open("20News_seedwords.json") as fp:
    seed_20news = json.load(fp)

In [5]:
#
data_nyt.head()
# data_nyt.iloc[0]["sentence"]

,sentence,label
0,"nasa, in preparation for a spacewalk on saturd...",science
1,if professional pride and strong defiance can ...,sports
2,"admittedly, the language is reconstructed and ...",arts
3,"palermo, sicily — roberta vinci beat top-seede...",sports
4,the argentine soccer club san lorenzo complete...,sports


In [6]:
#
data_20news.head() #newline characters visible

,sentence,label
0,from: (where's my thing)\nsubject: what car i...,rec
1,from: (guy kuo)\nsubject: si clock poll - fin...,comp
2,from: (thomas e willis)\nsubject: pb question...,comp
3,from: jgreen@amber (joe green)\nsubject: re: w...,comp
4,from: (jonathan mcdowell)\nsubject: re: shutt...,sci


In [7]:
#
seed_nyt #same format as 20News seed words

{'arts': ['music', 'orchestra', 'album', 'opera', 'ballet'],
 'business': ['companies', 'euro', 'economy', 'batteries', 'sales'],
 'science': ['space', 'researchers', 'scientists', 'research', 'science'],
 'sports': ['yankees', 'cup', 'league', 'basketball', 'golf'],
 'politics': ['republicans', 'senator', 'senate', 'democrats', 'election']}

In [8]:
seed_20news

{'alt': ['atheism', 'atheists', 'religion', 'objective'],
 'comp': ['graphics', 'windows', 'scsi', 'mac'],
 'misc': ['sale', 'offer', 'shipping', 'forsale'],
 'rec': ['car', 'bike', 'game', 'team'],
 'sci': ['encryption', 'circuit', 'candida', 'space'],
 'talk': ['turkish', 'gun', 'jews', 'armenian'],
 'soc': ['church', 'jesus', 'christ', 'christians']}

# Word2Vec

In [9]:
punctuation = set(string.punctuation)
stopwords = set(fe.text.ENGLISH_STOP_WORDS)

In [43]:
#tokenize
def tokenize(sentence):
    '''
    Returns a tokenized version of the given sentence, excluding punctuation and stopwords
    '''
    tokens = [w for w in (''.join([c for c in sentence.lower() if c not in punctuation]).split()) if w not in stopwords]
    return tokens

In [49]:
def word2vec(data, seeds):
    '''
    Return a DataFrame of the input with predicted labels using Word2Vec
    
    :param: data: a DataFrame with a "sentence" attribute
    :param: seeds: a dictionary of labels (keys) with a list of seed words (values)
    '''
    #reverse seeds dictionary
    genres = {}
    for genre,seed_words in seeds.items():
        for seed_word in seed_words:
            genres[seed_word] = genre
            
    data_ind = data.reset_index()
    
    data_tokens = []
    for row in data_ind.iterrows():
        data_tokens.append(tokenize(row[1]["sentence"]))

    model = Word2Vec(data_tokens, vector_size=100, sg=1)
    
#     label_reps = {}
#     for label in seeds:
#         label_reps[label] = model.wv.get_mean_vector(seeds[label])
    
    def predict_w2v(sentence):
        similarities = []
        for label in seeds:
            words_1 = seeds[label]
            words_2 = tokenize(sentence)
            cosine_sim = model.wv.n_similarity(words_1,words_2)
            similarities.append((cosine_sim,label))
        return max(similarities, key=lambda item: item[0])[1]
            
    df = data_ind.assign(prediction=data_ind["sentence"].apply(predict_w2v))
    
    return df

In [50]:
nyt_w2v = word2vec(data_nyt, seed_nyt)
nyt_w2v

,index,sentence,label,prediction
0,0,"nasa, in preparation for a spacewalk on saturd...",science,science
1,1,if professional pride and strong defiance can ...,sports,sports
2,2,"admittedly, the language is reconstructed and ...",arts,sports
3,3,"palermo, sicily — roberta vinci beat top-seede...",sports,sports
4,4,the argentine soccer club san lorenzo complete...,sports,sports
...,...,...,...,...
11522,11522,"melbourne, australia — after the tears and the...",sports,sports
11523,11523,"perth, australia — wimbledon semifinalist jerz...",sports,sports
11524,11524,while the nets are hoisting no. 5 to the barcl...,sports,sports
11525,11525,sydney — tennis australia chief executive stev...,sports,sports


In [59]:
nyt_w2v["label"].value_counts()

sports      8448
arts        1040
politics     977
business     973
science       89
Name: label, dtype: int64

In [52]:
nyt_w2v["prediction"].value_counts()

sports      8846
business    1103
science      801
politics     441
arts         336
Name: prediction, dtype: int64

In [53]:
micro_nyt_w2v = score_F1(nyt_w2v["label"], nyt_w2v["prediction"], "micro")
macro_nyt_w2v = score_F1(nyt_w2v["label"], nyt_w2v["prediction"], "macro")

print(micro_nyt_w2v)
print(macro_nyt_w2v)

0.8770712240825888
0.6159879225353991


In [55]:
w2v_20news = word2vec(data_20news, seed_20news)
w2v_20news

,index,sentence,label,prediction
0,0,from: (where's my thing)\nsubject: what car i...,rec,rec
1,1,from: (guy kuo)\nsubject: si clock poll - fin...,comp,comp
2,2,from: (thomas e willis)\nsubject: pb question...,comp,comp
3,3,from: jgreen@amber (joe green)\nsubject: re: w...,comp,comp
4,4,from: (jonathan mcdowell)\nsubject: re: shutt...,sci,sci
...,...,...,...,...
18254,18254,from: (stupendous man)\nsubject: re: temperat...,sci,sci
18255,18255,from: (jim smyton)\nsubject: re: monitors - s...,comp,comp
18256,18256,from: \nsubject: re: game length (was re: brav...,rec,rec
18257,18257,from: \nsubject: intel chmos 8086/8088 design...,misc,comp


In [56]:
micro_w2v_20news = score_F1(w2v_20news["label"], w2v_20news["prediction"], "micro")
macro_w2v_20news = score_F1(w2v_20news["label"], w2v_20news["prediction"], "macro")

print(micro_w2v_20news)
print(macro_w2v_20news)

0.7298866312503423
0.6620878770632501


In [58]:
w2v_20news["label"].value_counts()

comp    4685
sci     3879
rec     3822
talk    3180
soc      988
misc     912
alt      793
Name: label, dtype: int64

In [57]:
w2v_20news["prediction"].value_counts()

comp    5077
sci     4675
rec     3720
alt     1496
misc    1358
soc     1064
talk     869
Name: prediction, dtype: int64

In [41]:
# np.mean(Word2Vec(data_nyt.iloc[0]["sentence"],vector_size=100, sg=1).wv.vectors, axis=0)

array([-7.2530347e-05,  1.1783183e-03,  5.5800262e-04,  1.1176602e-03,
        2.2202190e-04, -7.4194855e-04,  1.0387886e-03,  2.0278418e-03,
       -1.7192799e-03, -1.4027825e-03, -4.1540153e-04, -1.5607105e-03,
        9.1590337e-04,  1.1275281e-03, -3.6987509e-05, -4.2349613e-05,
        2.6928489e-03, -2.5077426e-04, -8.5147988e-04, -3.5243083e-03,
        6.7180360e-04,  5.6308671e-04,  9.1066264e-04, -1.3493166e-04,
       -1.0023265e-03,  8.3123543e-04, -2.3133191e-03,  6.9141883e-04,
        3.7179564e-04,  7.0559140e-04,  1.8687396e-03, -7.5001974e-04,
       -1.4755287e-04, -1.7263208e-03, -6.0683081e-04,  7.8640931e-04,
        1.1886301e-03,  1.7178810e-04, -8.1940484e-04, -3.7080620e-04,
        8.6160708e-04, -1.0220795e-03, -2.5977448e-04,  8.8042446e-04,
        1.0401817e-04, -1.3923651e-03, -1.7017570e-03, -1.5260235e-03,
        9.6882856e-04,  4.6265117e-04,  6.0972292e-04, -1.2516815e-03,
       -8.0780283e-04,  4.3064945e-05, -6.6794385e-04,  4.8287612e-04,
      

In [42]:
# Word2Vec(data_nyt.iloc[0]["sentence"],vector_size=100, sg=1).transform(data_nyt.iloc[1]["sentence"])

AttributeError: 'Word2Vec' object has no attribute 'transform'

In [13]:
# data_nyt

,sentence,label
0,"nasa, in preparation for a spacewalk on saturd...",science
1,if professional pride and strong defiance can ...,sports
2,"admittedly, the language is reconstructed and ...",arts
3,"palermo, sicily — roberta vinci beat top-seede...",sports
4,the argentine soccer club san lorenzo complete...,sports
...,...,...
11522,"melbourne, australia — after the tears and the...",sports
11523,"perth, australia — wimbledon semifinalist jerz...",sports
11524,while the nets are hoisting no. 5 to the barcl...,sports
11525,sydney — tennis australia chief executive stev...,sports


# TF-IDF

## NYT

IR-TF-IDF treats the seed word set for each class as a query. The relevance of a document to a label is computed by aggregated TF-IDF values of its respective seed words. The label with the highest relevance is assigned to each document.

In [14]:
# genres_nyt = {}
# for genre,seed_words in seed_nyt.items():
#     for seed_word in seed_words:
#         genres_nyt[seed_word] = genre
        
# genres_nyt

In [15]:
# ir-tf-idf replication

# intialize vectorizer
# tfidf_model = fe.text.TfidfVectorizer(input="content", stop_words = {'english'})
# tfidf_vector = tfidf_model.fit_transform(data_nyt["sentence"])

In [16]:
#get vector with
# filtered = list(filter(lambda item: item[1] in genres_nyt, zip(tfidf_vector[0].toarray()[0].tolist(),tfidf_model.get_feature_names_out())))

In [17]:
# list(filtered)
# label_test = max(filtered, key=lambda item: item[0])
# label_test[1]

In [18]:
# nyt_tfidf = dict(zip(tfidf_model.get_feature_names_out()[i],tfidf_vector[].toarray()[0].tolist()[i]) for i in range(
#     len(tfidf_model.get_feature_names_out())))
# nyt_tfidf

In [19]:
# tfidf_model.get_feature_names_out()

In [20]:
def ir_tfidf(data, seeds):
    '''
    Return a DataFrame of the input with predicted labels using IR-TF-IDF
    
    :param: data: a DataFrame with "sentence" and "label" attributes
    :param: seeds: a dictionary of labels (keys) with a list of seed words (values)
    '''
    #reverse seeds dictionary
    genres = {}
    for genre,seed_words in seeds.items():
        for seed_word in seed_words:
            genres[seed_word] = genre
            
    data_ind = data.reset_index()
            
    model = fe.text.TfidfVectorizer(input="content", stop_words = {'english'})
    vector = model.fit_transform(data["sentence"])
    features = model.get_feature_names() #ADD _OUT FOR RECENT SKLEARN VERSION
    
    def predict(ind):
        filtered = filter(lambda item: item[1] in genres, zip(vector[ind].toarray()[0].tolist(),features))
        prediction = max(filtered, key=lambda item: item[0])
        return genres[prediction[1]]
    
    df = data_ind.assign(prediction = data_ind["index"].apply(predict))
    return df

In [21]:
# tfidf_nyt = ir_tfidf(data_nyt,seed_nyt)

In [22]:
# tfidf_nyt

In [23]:
# def score_F1(labels, predictions, averaging="micro"):
#     '''
#     Returns the micro F1 score given a list of labels and a list of predictions
    
#     :param: labels: a list of labels
#     :param: predictions: a list of predictions
#     :param: averaging: type of averaging method to use
#     '''
#     score = m.f1_score(labels, predictions, average=averaging)
#     return score

In [24]:
# micro_nyt_tfidf = score_F1(tfidf_nyt["label"], tfidf_nyt["prediction"], "micro")
# macro_nyt_tfidf = score_F1(tfidf_nyt["label"], tfidf_nyt["prediction"], "macro")

In [25]:
# micro_nyt_tfidf

In [26]:
# macro_nyt_tfidf

In [27]:
# tfidf_20News = ir_tfidf(data_20news,seed_20news)

In [28]:
# tfidf_20News